In [1]:
import pandas as pd
from os import listdir
import os
import glob

import re
#import all the negative reviews into dataframe
texts=[]
folder_path = 'neg'
for filename in glob.glob(os.path.join(folder_path, '*.txt')):
 with open(filename, 'rb') as f:
  text = f.read()
  texts.append(text)
df1=pd.DataFrame(texts)
df1.rename(columns={ df1.columns[0]: "review" }, inplace = True)
#sentiment 0 for negative reviews
df1['sentiment']=0

#import all positive reviews into dataframe
texts=[]
folder_path='pos'
for filename in glob.glob(os.path.join(folder_path, '*.txt')):
 with open(filename, 'rb') as f:
  text = f.read()
  texts.append(text)
df2=pd.DataFrame(texts)
df2.rename(columns={ df2.columns[0]: "review" }, inplace = True)
# sentiment 1 for positive reviews
df2['sentiment']=1
frames=[df1,df2]
df = pd.concat(frames)


df = df.reset_index()
#len(df)
df=df.drop(['index'], axis = 1)
df

,review,sentiment
0,b'plot : two teen couples go to a church party...,0
1,b'the happy bastard\'s quick movie review \nda...,0
2,"b""it is movies like these that make a jaded mo...",0
3,"b' "" quest for camelot "" is warner bros . \' f...",0
4,b'synopsis : a mentally unstable man undergoin...,0
...,...,...
1995,"b""wow ! what a movie . \nit's everything a mov...",1
1996,"b'richard gere can be a commanding actor , but...",1
1997,"b'glory--starring matthew broderick , denzel w...",1
1998,b'steven spielberg\'s second epic film on worl...,1


In [2]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
#remove all the unecessary symbols and tags from the text
def preprocessor(text):
   text =re.sub("<[^>]*>", "", text)
   emoticons = re.findall("(?::|;|=)(?:-)?(?:\)|\(|D|P)", text)
   text = re.sub("[\W]+", " " , text.lower()) + " ".join(emoticons).replace("-", "")
   return text

#split the text and make each word into tokens
def tokenizer(text):
   return text.split()

def tokenizer_stemmer(text):
 return[porter.stem(word) for word in text.split()]

In [3]:

import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
np.set_printoptions(precision=2)
tfidf = TfidfTransformer(use_idf=True, smooth_idf=True)

In [4]:
#vectorize all the reviews
df = df.sample(frac = 1) 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None,lowercase=True,preprocessor=preprocessor, tokenizer=tokenizer_stemmer,use_idf=True,norm="l2",smooth_idf=True)
y = df.sentiment.values
X = tfidf.fit_transform(df['review'])
len(df.review)

2000

In [5]:
from sklearn.model_selection import train_test_split
#logistic rfegression model with 5 fold cross validation
from sklearn.linear_model import LogisticRegressionCV
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.5, shuffle=False)
clf = LogisticRegressionCV(cv=5,scoring="accuracy",random_state=0,n_jobs=-1,verbose=3,max_iter=300).fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   43.3s remaining:  1.1min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   44.4s finished


In [6]:
clf.score(X_test,y_test)

0.813